In [1]:
# Huggingface ASR dataset to be tested
DATASET = 'google/fleurs'
LANGUAGE = 'en_us'
SPLIT = 'test'

# Whisper model name, can be one of the following: tiny/tiny.en/base/base.en/small/small.en/medium/medium.en
WHISPER_MODEL = 'tiny'

import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# Load the pre-trained OpenAI Whisper model
import whisper

model = whisper.load_model(WHISPER_MODEL).to(DEVICE)

In [ ]:
# Automatically download and load Huggingface dataset
import datasets
from datasets import load_dataset

dataset = load_dataset(DATASET, LANGUAGE, split=SPLIT)
dataset

In [ ]:
# Select a random sample from the testset and print the reference 
import random, os
idx = random.randint(0, len(dataset)-1)

# Prepare the path of the selected utterance
audio_path = os.path.join(os.path.dirname(dataset[idx]['path']), dataset[idx]['audio']['path'])
audio_path

In [ ]:
# The reference transcription for the utterance
print(f'ASR reference: "{dataset[idx]["transcription"]}"')

In [ ]:
# Load the utterance
from whisper.audio import load_audio

audio = load_audio(audio_path)
audio = torch.from_numpy(audio).to(DEVICE)

In [ ]:
# Whisepr transcription for the original speech signal
print(f'Default Whisper transcription: "{model.transcribe(audio)["text"]}"')

In [ ]:
# Load the relevant universal acoustic adversarial attack segment (0.64 seconds in length)
import numpy as np

loaded_array = np.load(f'audio_attack_segments/{WHISPER_MODEL}.np.npy')
audio_attack_segment = torch.from_numpy(loaded_array).to(audio.device)

In [ ]:
# Prepend the learned universal attack segment to the original speech signal
audio_with_prompts = torch.cat((audio_attack_segment, audio), dim=0)

In [ ]:
# Whisepr transcription for the concatenated speech signal
# Whisper is "muted" in this case
print(f'Whisper transcription with the learned attack segment: "{model.transcribe(audio_with_prompts)["text"]}"')